## Deliverable 3: Optimize the Model 

In [667]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import sklearn as skl
import tensorflow as tf

In [668]:
# Read the charity_data.csv.
application_df1 = pd.read_csv("resources/charity_data.csv")
application_df1.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [669]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df1.drop(['EIN','NAME'],axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [670]:
APPLICATION_TYPE_counts=application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace
replace_types = list(APPLICATION_TYPE_counts[APPLICATION_TYPE_counts < 1000].index)

# Replace in DataFrame
for typ in replace_types:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(typ,"Other")


# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [671]:
CLASSIFICATION_counts=application_df.CLASSIFICATION.value_counts()

# Determine which values to replace
replace_classes = list(CLASSIFICATION_counts[CLASSIFICATION_counts < 2000].index)

# Replace in DataFrame
for clas in replace_classes:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(clas,"Other")


# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [672]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [673]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [674]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,axis=1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [675]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [676]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [677]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 35

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_154 (Dense)           (None, 90)                3510      
                                                                 
 dense_155 (Dense)           (None, 35)                3185      
                                                                 
 dense_156 (Dense)           (None, 1)                 36        
                                                                 
Total params: 6,731
Trainable params: 6,731
Non-trainable params: 0
_________________________________________________________________


In [678]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [679]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [680]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=5,
    save_weights_only=True,
    save_freq='epoch')

In [681]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=150,callbacks=[cp_callback])

Epoch 1/150
783/804 [============================>.] - ETA: 0s - loss: 48825.9648 - accuracy: 0.4966
Epoch 1: saving model to checkpoints/weights.01.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 47872.2031 - accuracy: 0.4970
Epoch 2/150
790/804 [============================>.] - ETA: 0s - loss: 25427.6816 - accuracy: 0.4947
Epoch 2: saving model to checkpoints/weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 25015.1211 - accuracy: 0.4942
Epoch 3/150
783/804 [============================>.] - ETA: 0s - loss: 29724.9570 - accuracy: 0.4947
Epoch 3: saving model to checkpoints/weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 29029.3223 - accuracy: 0.4939
Epoch 4/150
794/804 [============================>.] - ETA: 0s - loss: 8372.0020 - accuracy: 0.4919
Epoch 4: saving model to checkpoints/weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 8415.7559 - accuracy: 0.4927
Epoch 5/150
80

Epoch 35/150
788/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5330
Epoch 35: saving model to checkpoints/weights.35.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 36/150
793/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5328
Epoch 36: saving model to checkpoints/weights.36.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 37/150
799/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5324
Epoch 37: saving model to checkpoints/weights.37.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 38/150
781/804 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5320
Epoch 38: saving model to checkpoints/weights.38.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 39/150
800/804 [==============

Epoch 69/150
790/804 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5316
Epoch 69: saving model to checkpoints/weights.69.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 70/150
788/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5331
Epoch 70: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 71/150
799/804 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5318
Epoch 71: saving model to checkpoints/weights.71.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 72/150
785/804 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5319
Epoch 72: saving model to checkpoints/weights.72.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 73/150
791/804 [==============

Epoch 103/150
798/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5320
Epoch 103: saving model to checkpoints/weights.103.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 104/150
783/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5327
Epoch 104: saving model to checkpoints/weights.104.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 105/150
784/804 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5320
Epoch 105: saving model to checkpoints/weights.105.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 106/150
785/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5324
Epoch 106: saving model to checkpoints/weights.106.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 107/150
800/804 [=

804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 137/150
789/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5328
Epoch 137: saving model to checkpoints/weights.137.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 138/150
784/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5328
Epoch 138: saving model to checkpoints/weights.138.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 139/150
798/804 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5319
Epoch 139: saving model to checkpoints/weights.139.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 140/150
789/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5320
Epoch 140: saving model to checkpoints/weights.140.hdf5
804/804 [===============

In [682]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7996 - accuracy: 0.6584 - 443ms/epoch - 2ms/step
Loss: 0.7995995283126831, Accuracy: 0.6584256291389465


In [683]:
# Restore the model weights
nn.load_weights("checkpoints/weights.100.hdf5")

# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

**Cheking if it works**

In [684]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('AlphabetSoupCharity_Optimization.h5')
# Evaluate the completed model using the test data

model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8041 - accuracy: 0.6575 - 480ms/epoch - 2ms/step
Loss: 0.8040558695793152, Accuracy: 0.6574926972389221
